**Install the 'spacy' library and the the 'spacy_transformers' library**

In [ ]:
!pip install -U spacy
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.4 MB/s eta 0:00:00


**Import necessary libraries**

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

**Load the dataset**

In [ ]:
cv_data=json.load(open('/content/drive/MyDrive/JobDescription/dataset/train_data.json','r'))

**Initialize a spaCy configuration file by filling in values from 'base_config.cfg' and save the resulting configuration to 'config.cfg'**

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/JobDescription/config/base_config.cfg /content/drive/MyDrive/JobDescription/config/config.cfg

2023-09-17 13:35:26.981471: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/JobDescription/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


**Process text data using spaCy**

In [ ]:
def get_spacy_doc(file,data):
  nlp=spacy.blank("en")
  db=DocBin()
  for text,annot  in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']
    ents=[]
    entity_indices=[]
    for start,end,label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue
      entity_indices=entity_indices+list(range(start,end))
      try:
        span=doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue
      if span is None:
        err_data=str([start,end])+"     "+str(text)+"\n"
        file.write(err_data)
      else:
        ents.append(span)
    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  return db

**Split the dataset 'cv_data' into training and testing sets using train_test_split with a test size of 30% (0.3), assigning the results to 'train' (training set) and 'test' (testing set)**

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.3)

**Load the spaCy model from the specified path**

In [ ]:
nlp=spacy.load('/content/drive/MyDrive/JdModel/JdModel/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


**Install the 'PyMuPDF' library for working with PDF file**

In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 17.0 MB/s eta 0:00:00


**Open train_file.txt for writing. Preprocess and convert the 'train' and 'test' data into spaCy format, and save them as spaCy binary files**

In [ ]:
file=open('/content/drive/MyDrive/JobDescription/JobDescriptionModel/train_file.txt','w')
db=get_spacy_doc(file,train)
db.to_disk('/content/drive/MyDrive/JobDescription/JobDescriptionModel/train_data.spacy')
db=get_spacy_doc(file,test)
db.to_disk('/content/drive/MyDrive/JobDescription/JobDescriptionModel/test_data.spacy')
file.close()

100%|██████████| 60/60 [00:01<00:00, 47.13it/s]


**Train the spaCy model using the specified configuration file, training and development data and GPU with ID 0 and save the trained model to the specified output directory**

In [ ]:
!python -m spacy train /content/drive/MyDrive/JobDescription/config/config.cfg --output /content/drive/MyDrive/JobDescription/JobDescriptionModel/output --paths.train  /content/drive/MyDrive/JobDescription/JobDescriptionModel/train_data.spacy --paths.dev /content/drive/MyDrive/JobDescription/JobDescriptionModel/test_data.spacy --gpu-id 0

2023-09-17 13:38:08.449770: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory:
/content/drive/MyDrive/JobDescription/JobDescriptionModel/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weigh